In [1]:
import pandas as pd
df_songs = pd.read_csv("songs_details.csv")

In [7]:
df_songs.columns

Index(['song_id', 'song_name', 'song_artist', 'searched_lyrics', 'lyrics_link',
       'lyrics', 'searched_spotify', 'popularity', 'release_date',
       'duration_ms', 'explicit', 'artists', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'searched_lastfm', 'lastfm_count',
       'lastfm_wiki', 'searched_artists', 'first_artist', 'artist_genres',
       'artist_popularity'],
      dtype='object')

In [6]:
print(df_songs["popularity"].mean())
print(df_songs["artist_popularity"].mean())

42.800942285041224
70.24852767962308
